In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tqdm.notebook import tqdm
import glob
import numpy as np
from utils_analysis import *
from PIL import Image
#import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

In [2]:
dataset_dir = "../dataset/animal_parts_dataset/preprocessed_data/resized_images_val"
#dataset_dir = "../dataset/animal_parts_dataset/preprocessed_data/resized_images"
training_methods = ["dino"]#, "supervised"] 
depth_list = [8, 12]
num_models = 6
num_heads = 6
patch_size = 16
blur_size = patch_size * 2
batch_size = 32

transform = pth_transforms.Compose([
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [3]:
save_dir = "../dataset/animal_parts_dataset/preprocessed_data/"

In [4]:
image_path_list = sorted(glob.glob(f"{dataset_dir}/*.png"))
dataset = ImageDataset(image_path_list, transform)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

In [5]:
num_images = len(image_path_list)

In [6]:
num_images

1067

In [7]:
res_dict = {}
res_dict["info"] = ["num_models", "depth", "num_head+mean", "num_images", "xy"]
for tm in training_methods:
    res_dict[tm] = {}
    for depth in depth_list:
        print(tm, depth)
        gaze_pos_model_all = np.zeros((num_models, depth, num_heads+1, num_images, 2))
        for trial_idx in tqdm(range(num_models)):
            model, device = model_load(tm, trial_idx+1, depth, patch_size)
            gaze_pos_model_all[trial_idx] = get_gaze_pos_model_dataset(model, device, dataloader, patch_size, blur_size)
            del model
            torch.cuda.empty_cache()
        res_dict[tm][str(depth)] = gaze_pos_model_all
np.savez_compressed(f"{save_dir}/vit_gaze_pos_val.npz", **res_dict)

dino 8


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]